In [1]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import os
import pandas as pd
import numpy as np
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm

In [3]:
# #GPU 설정 할 때마다 쓸 지피유로 변경
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2, 3"

In [4]:
#새로운버젼2
class SFCN(nn.Module):
    def __init__(self, channel_number=[32, 64, 128, 256, 256, 64], output_dim=1, dropout=True):
        super(SFCN, self).__init__()
        n_layer = len(channel_number)
        self.feature_extractor = nn.Sequential()
        for i in range(n_layer):
            if i == 0:
                in_channel = 1
            else:
                in_channel = channel_number[i-1]
            out_channel = channel_number[i]
            if i < n_layer-1:
                self.feature_extractor.add_module('conv_%d' % i,
                                                  self.conv_layer(in_channel,
                                                                  out_channel,
                                                                  maxpool=True,
                                                                  kernel_size=3,
                                                                  padding=1))
            else:
                self.feature_extractor.add_module('conv_%d' % i,
                                                  self.conv_layer(in_channel,
                                                                  out_channel,
                                                                  maxpool=False,
                                                                  kernel_size=1,
                                                                  padding=0))
        avg_shape = [5, 6, 5]
        self.classifier = nn.Sequential()
        self.classifier.add_module('average_pool', nn.AvgPool3d(avg_shape))
        if dropout is True:
            self.classifier.add_module('dropout', nn.Dropout(0.5))
        i = n_layer
        in_channel = channel_number[-1]
        out_channel = output_dim  
        self.classifier.add_module('last_conv',
                                   nn.Conv3d(in_channel, out_channel, padding=0, kernel_size=1))
        
        self.final_fc = nn.Linear(1, output_dim)

    @staticmethod
    def conv_layer(in_channel, out_channel, maxpool=True, kernel_size=3, padding=0, maxpool_stride=2):
        if maxpool is True:
            layer = nn.Sequential(
                nn.Conv3d(in_channel, out_channel, padding=padding, kernel_size=kernel_size),
                nn.BatchNorm3d(out_channel),
                nn.MaxPool3d(2, stride=maxpool_stride),
                nn.ReLU(),
            )
        else:
            layer = nn.Sequential(
                nn.Conv3d(in_channel, out_channel, padding=padding, kernel_size=kernel_size),
                nn.BatchNorm3d(out_channel),
                nn.ReLU()
            )
        return layer
    
    def forward(self, x):
        x_f = self.feature_extractor(x)
        x = self.classifier(x_f)
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.final_fc(x))  # Add sigmoid activation function
        x = x * 70 + 20  # Scale the output between 20 and 90
        return x


In [5]:
class MRIDataset(Dataset):
    def __init__(self, data_dir, csv_file, transform=None):
        self.data_dir = data_dir
        self.data_info = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, f"{self.data_info.iloc[idx, 0]}_T1w_restore_brain.nii.gz")
        image = nib.load(img_name).get_fdata()

        # 이미지를 3D로 변환 (첫 번째 차원 추가)
        image = image[np.newaxis]

        # 문자열로 저장되어 있는 나이 정보를 숫자로 변환
        age = float(self.data_info.iloc[idx, 1])
        
        subject_id = self.data_info.iloc[idx, 0]  # Subject ID 가져오기
        gender = self.data_info.iloc[idx, 2]
        
        if self.transform:
            image = self.transform(image)

        return image, age, gender, subject_id  # Subject ID와 Gender 반환

In [6]:
class DataScaler(object):
    def __call__(self, image):
        image = image / image.mean()  # 평균으로 스케일링
        image = torch.tensor(image, dtype=torch.float32)  # 이미지를 텐서로 변환
        return image

In [7]:
#이거 새로 정의한거
def mean_absolute_error(y_pred, y_true):
    y_true = y_true.to(y_pred.device)  # y_true도 y_pred와 동일한 device로 이동
    return torch.mean(torch.abs(y_pred - y_true))

In [8]:
# GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [9]:
# 데이터 로딩 및 전처리
data_folder = '../hcp_ya/ya_resize'
csv_file = 'subject_age_gender.csv'

데이터 로드

In [14]:
# 데이터 전처리를 위한 변환기 적용
transform = transforms.Compose([DataScaler()])
dataset = MRIDataset(data_folder, csv_file, transform=transform)

# 데이터를 train, val 세트로 나누기
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# DataLoader 설정
batch_size = 4
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

test data

In [15]:
batch_size = 4
transform = transforms.Compose([DataScaler()])
test_data_folder = 'test' #테스트nii.gz 있는 경로
test_csv_file = 'test.csv' #테스트 csv 파일 경로
# 따로 준비한 Test 데이터 로딩
test_dataset = MRIDataset(test_data_folder, test_csv_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
# 모델 초기화
model = SFCN(output_dim=1).to(device)
#model = torch.nn.DataParallel(model, device_ids=[2, 3])  # 여러 GPU에서 모델을 병렬로 실행하도록 설정합니다.

# 가중치 로드하기
path = "run_20190719_00_epoch_best_mae.p"  # 가중치 파일 경로
pretrained_dict = torch.load(path, map_location=device)

# 모델의 state_dict 가져오기
model_dict = model.state_dict()

# 모델과 가중치 사이즈 맞추기
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #이거 더 적게 해보기

best_val_loss = float('inf')  # 최고의 검증 손실 값을 저장하기 위한 변수 초기화

num_epochs = 1
train_losses = []
val_losses = []

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 12.00 GiB total capacity; 11.29 GiB already allocated; 0 bytes free; 11.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정

    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}, Train Loss: ", leave=False):
    #for batch in train_loader:
        inputs = batch[0].to(device)
        labels = torch.tensor(batch[1], dtype=torch.float32).view(-1, 1).to(device)


        outputs = model(inputs)
        loss = mean_absolute_error(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_loss:.4f}")

    # Validation
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        total_val_loss = 0.0
        for val_batch in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: ", leave=False):
        #for val_batch in val_loader:
            val_inputs = val_batch[0].to(device)
            val_labels = torch.tensor(val_batch[1], dtype=torch.float32).unsqueeze(1).to(device)

            val_outputs = model(val_inputs)
            val_loss = mean_absolute_error(val_outputs, val_labels)

            total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")

        # 가중치 저장 (Validation Loss가 가장 낮을 때에만 저장)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "sfcn.p")
            print("Best model saved.")

In [ ]:
subject_ids = []
true_ages = []
predicted_ages = []
genders = []

In [ ]:
# Test loop
model.eval()
with torch.no_grad():
    total_test_loss = 0.0
    for test_batch in test_loader:
        for i in range(test_batch[0].size(0)):
            test_input = test_batch[0][i].unsqueeze(0).to(device)
            test_label = torch.tensor([test_batch[1][i]], dtype=torch.float32).unsqueeze(1).to(device)
            test_subject_id = test_batch[2][i]
            test_gender = test_batch[3][i]

            test_output = model(test_input)
            test_loss = mean_absolute_error(test_output, test_label)
            total_test_loss += test_loss.item()

            # Append subject ID, true age, predicted age, and gender to the lists
            subject_ids.append(test_subject_id)
            true_ages.append(test_label.item())
            predicted_ages.append(test_output.item())  # Change this line
            genders.append(test_gender)
            
    avg_test_loss = total_test_loss / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}")

with open("run_20190719_00_epoch_best_mae.csv", mode="w", newline="") as file:
    fieldnames = ["SubjectID", "TrueAge", "PredictedAge", "Gender"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()

    for subject, true_age, predicted_age, gender in zip(subject_ids, true_ages, predicted_ages, genders):
        # If subject is stored as (Tensor) in the list of test_subject_ids,
        # convert the subject to string and remove "tensor" from the Subject ID
        if isinstance(subject, torch.Tensor):
            subject = subject.item()
        
        # Extract the predicted age value
        predicted_age_scalar = predicted_age
        
        writer.writerow({"SubjectID": gender, "TrueAge": true_age, "PredictedAge": predicted_age_scalar, "Gender": subject})
